

## 🎯 **Overview**

This system automatically generates optimized weekly work schedules for café operations using advanced constraint satisfaction and optimization techniques. It ensures fair workload distribution, role-based coverage, and business rule compliance.

## ✨ **Key Features**

- 🎯 **Role-based scheduling** (MANAGER, BARISTA, WAITER, SANDWICH)
- ⚖️ **Fair workload distribution** with hours policy enforcement
- 📅 **Automatic weekend coverage** with manager supervision
- 🕐 **Flexible time windows** (early SANDWICH prep, staggered shifts)
- 🔄 **Synthetic week generation** for future planning
- 📊 **Comprehensive reporting** with validation
- 🛡️ **Constraint satisfaction** (no overlaps, hours caps, role eligibility)

## 🏗️ **Architecture**

- **Algorithm**: Greedy optimization with backtracking
- **Constraints**: Hard (no overlaps, hours caps) + Soft (fairness, targets)
- **Configuration**: YAML-based business rules
- **Validation**: Multi-layer constraint checking
- **Output**: CSV exports with detailed employee information

---


## 🚀 **Quick Start**

### **Step 1: Configuration**
Set your parameters below and run the cells in sequence.


In [ ]:
# ============================================================================
# CONFIGURATION
# ============================================================================

# Import required libraries
import pandas as pd
import datetime as dt
import sys
import importlib
from pathlib import Path

# Import scheduler modules
from scheduler.config import load_config
from scheduler.data_io import read_employees, read_shifts, write_assignments
from scheduler.engine_baseline import greedy_schedule, build_requirements_for_day
from scheduler.validator import validate_assignments, summarize_assignments

print("📦 All modules imported successfully!")
print("🔧 Scheduler system ready for configuration...")


In [ ]:
# ============================================================================
# PARAMETERS - Customize these for your needs
# ============================================================================

# Week to generate (ISO format: YYYY-W##)
WEEK_ID = "2025-W48"  # Change this to generate different weeks

# File paths
EMPLOYEES_CSV = "./data/employees_new_12w_v2.csv"
SHIFTS_CSV = "./data/shift_new_12w_v2.csv"
CONFIG_PATH = "./scheduler_config.yaml"
OUTPUT_CSV = f"./schedule_{WEEK_ID.replace('-W', '_week_')}.csv"
COMPREHENSIVE_CSV = f"./comprehensive_schedule_{WEEK_ID.replace('-W', '_week_')}.csv"

print("⚙️  Configuration Parameters:")
print(f"   📅 Week: {WEEK_ID}")
print(f"   👥 Employees: {EMPLOYEES_CSV}")
print(f"   📋 Shifts: {SHIFTS_CSV}")
print(f"   ⚙️  Config: {CONFIG_PATH}")
print(f"   📄 Output: {OUTPUT_CSV}")
print(f"   📊 Detailed: {COMPREHENSIVE_CSV}")
print()
print("✅ Ready to generate schedule!")


### **Step 2: Load Data & Configuration**


In [ ]:
# ============================================================================
# LOAD CONFIGURATION
# ============================================================================

# Load and display configuration
cfg = load_config(CONFIG_PATH)

print("🔧 Configuration Loaded Successfully!")
print("=" * 60)
print(f"🌍 Timezone: {cfg.timezone}")
print(f"⏰ Default Shift: {cfg.default_shift.start} - {cfg.default_shift.end}")
print(f"📊 Hours Policy:")
for role, policy in cfg.hours_policy.items():
    target = f"{policy['target_min']}-{policy['target_max']}h"
    hard_cap = f"{policy['hard_cap']}h"
    print(f"   {role}: Target {target}, Hard Cap {hard_cap}")

print(f"\n📅 Weekend Requirements:")
for role, count in cfg.weekend_requirements.items():
    print(f"   {role}: {count} per weekend day")

print(f"\n✅ Configuration validated and ready!")


In [ ]:
# ============================================================================
# LOAD EMPLOYEE DATA
# ============================================================================

# Load employees data
employees = read_employees(EMPLOYEES_CSV)

print("👥 Employee Data Loaded Successfully!")
print("=" * 60)
print(f"📊 Total Employees: {len(employees)}")
print(f"📋 Role Distribution:")
role_counts = employees['primary_role'].value_counts()
for role, count in role_counts.items():
    print(f"   {role}: {count} employees")

print(f"\n👤 Employee List:")
for _, emp in employees.iterrows():
    name = f"{emp['first_name']} {emp['last_name']}"
    role = emp['primary_role']
    print(f"   {name} (ID: {emp['employee_id']}) - {role}")

print(f"\n✅ All employees loaded and ready for scheduling!")


In [ ]:
# ============================================================================
# LOAD OR CREATE SHIFTS
# ============================================================================

# Load shifts for the target week
shifts = read_shifts(SHIFTS_CSV, week_id=WEEK_ID)

if shifts.empty:
    print(f"📅 Week {WEEK_ID} not found in database")
    print("🔄 Auto-creating synthetic shifts for the target week...")
    
    # Generate dates for the target week
    year = int(WEEK_ID.split('-W')[0])
    week = int(WEEK_ID.split('-W')[1])
    dates = [dt.date.fromisocalendar(year, week, dow) for dow in range(1, 8)]
    
    # Create unique shift IDs
    base_id = 100000 + (year % 1000) * 100 + week
    shifts = pd.DataFrame({
        'shift_id': [base_id + i for i in range(7)],
        'date': dates,
        'week_id': [WEEK_ID] * 7,
    })
    
    print(f"✅ Created {len(shifts)} synthetic shifts")
    print(f"📅 Dates: {[d.strftime('%Y-%m-%d (%A)') for d in dates]}")
else:
    print(f"📅 Found existing shifts for {WEEK_ID}")
    print(f"📊 {len(shifts)} shifts loaded")

print(f"\n✅ Shifts ready for scheduling!")


### **Step 3: Generate Schedule**


In [ ]:
# ============================================================================
# SCHEDULE GENERATION
# ============================================================================

print("🚀 Generating optimized schedule...")
print("=" * 60)

# Show daily requirements
print("📋 Daily Requirements:")
for date in sorted(shifts["date"].unique()):
    ds = pd.Timestamp(date).strftime("%Y-%m-%d")
    req = build_requirements_for_day(ds, cfg)
    day_name = pd.Timestamp(date).day_name()
    print(f"   {ds} ({day_name}): {req}")

print(f"\n🎯 Starting schedule generation...")

try:
    # Generate assignments using advanced algorithm
    assignments = greedy_schedule(employees, shifts, cfg)
    
    print(f"✅ Schedule Generated Successfully!")
    print(f"📊 Total Assignments: {len(assignments)}")
    
    # Save basic schedule
    write_assignments(OUTPUT_CSV, assignments)
    print(f"💾 Basic schedule saved to: {OUTPUT_CSV}")
    
except RuntimeError as e:
    print(f"❌ Schedule generation failed: {e}")
    print("🔧 This might be due to:")
    print("   • Insufficient staff for requirements")
    print("   • Conflicting constraints")
    print("   • Hours policy too restrictive")
    assignments = None


### **Step 4: Validation & Quality Assurance**


In [ ]:
# ============================================================================
# VALIDATION
# ============================================================================

# Force reload validator module to pick up fixes
if 'scheduler.validator' in sys.modules:
    importlib.reload(sys.modules['scheduler.validator'])
from scheduler.validator import validate_assignments

if assignments is not None and not assignments.empty:
    print("🔍 Validating generated schedule...")
    print("=" * 60)
    
    # Build requirements for validation
    req_by_date = {}
    for date in sorted(shifts["date"].unique()):
        ds = pd.Timestamp(date).strftime("%Y-%m-%d")
        req_by_date[ds] = build_requirements_for_day(ds, cfg)
    
    try:
        # Comprehensive validation
        validate_assignments(
            employees,
            shifts,
            assignments,
            start_hm=cfg.default_shift.start,
            end_hm=cfg.default_shift.end,
            requirements_by_date=req_by_date,
        )
        
        print("✅ Validation Passed!")
        print("   ✓ No employee overlaps")
        print("   ✓ All coverage requirements met")
        print("   ✓ Hours caps respected")
        print("   ✓ Role eligibility verified")
        print("   ✓ Café hours compliance")
        
    except Exception as e:
        print(f"❌ Validation Failed: {e}")
        print("🔧 Please check the error details above")
        
else:
    print("⚠️ No assignments to validate")


### **Step 5: Comprehensive Reporting**


In [ ]:
# ============================================================================
# COMPREHENSIVE SCHEDULE REPORT
# ============================================================================

if assignments is not None and not assignments.empty:
    print("📊 Creating comprehensive schedule report...")
    print("=" * 60)
    
    # Merge with employee data for detailed report
    detailed_assignments = assignments.merge(
        employees[['employee_id', 'first_name', 'last_name', 'primary_role', 
                  'skill_coffee', 'skill_sandwich', 'customer_service_rating', 'skill_speed']], 
        left_on='emp_id', right_on='employee_id', how='left'
    )
    
    # Process timestamps and calculate hours
    detailed_assignments['start_time'] = pd.to_datetime(detailed_assignments['start_time'])
    detailed_assignments['end_time'] = pd.to_datetime(detailed_assignments['end_time'])
    detailed_assignments['shift_hours'] = (
        detailed_assignments['end_time'] - detailed_assignments['start_time']
    ).dt.total_seconds() / 3600
    detailed_assignments['day_of_week'] = detailed_assignments['start_time'].dt.day_name()
    detailed_assignments['date'] = detailed_assignments['start_time'].dt.date
    
    # Reorder columns for better readability
    column_order = [
        'shift_id', 'emp_id', 'first_name', 'last_name', 'primary_role',
        'date', 'day_of_week', 'start_time', 'end_time', 'shift_hours',
        'skill_coffee', 'skill_sandwich', 'customer_service_rating', 'skill_speed'
    ]
    detailed_assignments = detailed_assignments[column_order]
    detailed_assignments = detailed_assignments.sort_values(['date', 'start_time', 'emp_id'])
    
    # Save comprehensive report
    detailed_assignments.to_csv(COMPREHENSIVE_CSV, index=False)
    
    print(f"✅ Comprehensive report saved to: {COMPREHENSIVE_CSV}")
    print(f"📊 Total assignments: {len(detailed_assignments)}")
    print(f"📋 Columns: {', '.join(column_order)}")
    
else:
    print("⚠️ No assignments to create comprehensive report")


### **Step 6: Employee Hours Analysis**


In [ ]:
# ============================================================================
# EMPLOYEE HOURS ANALYSIS
# ============================================================================

if assignments is not None and not assignments.empty:
    print("📊 Analyzing employee hours and workload distribution...")
    print("=" * 60)
    
    # Calculate total hours per employee
    employee_hours = detailed_assignments.groupby(['emp_id', 'first_name', 'last_name', 'primary_role']).agg({
        'shift_hours': 'sum',
        'shift_id': 'count'
    }).rename(columns={'shift_id': 'total_shifts'}).reset_index()
    
    # Sort by role and hours
    employee_hours = employee_hours.sort_values(['primary_role', 'shift_hours'], ascending=[True, False])
    
    print("👥 EMPLOYEE HOURS SUMMARY")
    print("=" * 60)
    
    # Show hours by role with status indicators
    for role in ['MANAGER', 'BARISTA', 'WAITER', 'SANDWICH']:
        role_employees = employee_hours[employee_hours['primary_role'] == role]
        if not role_employees.empty:
            print(f"\n🔹 {role} ROLE:")
            print("-" * 40)
            
            # Get role policy for comparison
            role_policy = cfg.hours_policy.get(role, {})
            target_min = role_policy.get('target_min', 0)
            target_max = role_policy.get('target_max', 40)
            hard_cap = role_policy.get('hard_cap', 40)
            
            for _, emp in role_employees.iterrows():
                hours = emp['shift_hours']
                shifts = emp['total_shifts']
                name = f"{emp['first_name']} {emp['last_name']}"
                
                # Determine status
                if target_min <= hours <= target_max:
                    status = "✅ Within target"
                elif hours < target_min:
                    status = f"⚠️ Below target (need {target_min}h)"
                else:
                    status = f"⚠️ Above target (max {target_max}h)"
                
                if hours > hard_cap:
                    status = f"❌ OVER HARD CAP ({hard_cap}h)"
                
                print(f"   {name}: {hours:.1f}h ({shifts} shifts) - {status}")
    
    # Overall summary
    print(f"\n📈 OVERALL SUMMARY")
    print("=" * 60)
    total_hours = employee_hours['shift_hours'].sum()
    avg_hours = employee_hours['shift_hours'].mean()
    min_hours = employee_hours['shift_hours'].min()
    max_hours = employee_hours['shift_hours'].max()
    
    print(f"📊 Total hours across all employees: {total_hours:.1f}h")
    print(f"📊 Average hours per employee: {avg_hours:.1f}h")
    print(f"📊 Hours range: {min_hours:.1f}h - {max_hours:.1f}h")
    
    # Check for violations
    over_cap = []
    for _, emp in employee_hours.iterrows():
        role = emp['primary_role']
        hours = emp['shift_hours']
        role_policy = cfg.hours_policy.get(role, {})
        hard_cap = role_policy.get('hard_cap', 40)
        
        if hours > hard_cap:
            over_cap.append(f"{emp['first_name']} {emp['last_name']} ({role}): {hours:.1f}h > {hard_cap}h")
    
    if over_cap:
        print(f"\n⚠️ EMPLOYEES OVER HARD CAP:")
        for emp in over_cap:
            print(f"   • {emp}")
    else:
        print(f"\n✅ All employees within their hard caps")
        
else:
    print("⚠️ No assignments to analyze")


### **Step 7: Daily Schedule View**


In [ ]:
# ============================================================================
# DAILY SCHEDULE VIEW
# ============================================================================

if assignments is not None and not assignments.empty:
    print("📅 Daily Schedule Overview")
    print("=" * 60)
    
    # Group by date to show daily coverage
    for date in sorted(detailed_assignments['start_time'].dt.date.unique()):
        day_assignments = detailed_assignments[detailed_assignments['start_time'].dt.date == date]
        day_name = day_assignments.iloc[0]['start_time'].strftime('%A')
        date_str = day_assignments.iloc[0]['start_time'].strftime('%Y-%m-%d')
        
        print(f"\n📅 {date_str} ({day_name})")
        print("-" * 40)
        
        # Group by role
        for role in sorted(day_assignments['primary_role'].unique()):
            role_assignments = day_assignments[day_assignments['primary_role'] == role]
            print(f"   🔹 {role}: {len(role_assignments)} person(s)")
            
            for _, shift in role_assignments.iterrows():
                name = f"{shift['first_name']} {shift['last_name']}"
                start_time = shift['start_time'].strftime('%H:%M')
                end_time = shift['end_time'].strftime('%H:%M')
                shift_hours = shift['shift_hours']
                print(f"      • {name}: {start_time}-{end_time} ({shift_hours:.1f}h)")
    
    print(f"\n✅ Daily schedule view complete!")
    
else:
    print("⚠️ No assignments to display")


### **Step 8: Final Summary & Next Steps**


In [ ]:
# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("🎉 SCHEDULING COMPLETE!")
print("=" * 60)

if assignments is not None and not assignments.empty:
    print("✅ SUCCESS!")
    print(f"   📅 Week: {WEEK_ID}")
    print(f"   📊 Assignments: {len(assignments)}")
    print(f"   👥 Employees: {len(employees)}")
    print(f"   📄 Basic Schedule: {OUTPUT_CSV}")
    print(f"   📊 Comprehensive Report: {COMPREHENSIVE_CSV}")
    
    # Show role distribution
    role_dist = detailed_assignments['primary_role'].value_counts()
    print(f"\n📈 Role Distribution:")
    for role, count in role_dist.items():
        print(f"   {role}: {count} assignments")
    
    print(f"\n🎯 Next Steps:")
    print(f"   1. 📋 Review the comprehensive schedule CSV")
    print(f"   2. ⚖️ Check employee hours are balanced")
    print(f"   3. 🔧 Adjust configuration if needed")
    print(f"   4. 📤 Export to your scheduling system")
    print(f"   5. 📧 Share schedule with team")
    
    print(f"\n💡 Tips:")
    print(f"   • Change WEEK_ID to generate different weeks")
    print(f"   • Modify scheduler_config.yaml for business rules")
    print(f"   • Use comprehensive CSV for detailed analysis")
    
else:
    print("❌ SCHEDULING FAILED")
    print(f"\n🔧 Troubleshooting:")
    print(f"   1. Check if you have enough employees for each role")
    print(f"   2. Verify hours policy allows sufficient coverage")
    print(f"   3. Try reducing requirements in config")
    print(f"   4. Check if any employees are over their hard caps")

print(f"\n📝 Configuration: {CONFIG_PATH}")
print(f"📅 Generated for: {WEEK_ID}")
print("=" * 60)
print("🚀 AI-Assisted Café Rostering System - Complete!")
